## Data Fetch

In [54]:
import requests
import json
import os

def crtFold(folder):
    path = folder + '/'
    isExists=os.path.exists(path)
    if(not isExists):
        os.mkdir(path)
    return path
for i in range(1,7):
    key = '1N2)WAQaEFzyJP*F)Al2Kw(('
    url = 'http://api.stackexchange.com/2.2/questions?key={}&page={}&pagesize=100\
    &fromdate=1472688000&todate=1475193600&order=desc&sort=creation&tagged=python%3Bpandas&site=stackoverflow'.format(key,i)
    r = requests.get(url)
    # print(len((r.json())['items']))
    with open(crtFold('data') + 'dataFetch{}.json'.format(i), 'w') as f:
        f.write(json.dumps(r.json(),indent = 4))

## Questions UserId

In [93]:
import json
import csv
import os
question_userid = list()
user_ids = list()
user_ques_dict = {}
ques_user_dict = {}
listJsonFiles = os.listdir("data")
for file in listJsonFiles:
    with open("data/" + file) as jsonfile:
        jsonData = json.load(jsonfile)
        for item in jsonData["items"]:
            user_ids.append(item['owner']['user_id'])
            question_userid.append((item['owner']['user_id'], item['title']))
            user_ques_dict[item['owner']['user_id']] = item['title']
            ques_user_dict[item['title']] = item['owner']['user_id']
with open('question_userid.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["user_id", "question title"])
    writer.writerows(question_userid)
    

## Badges Counts

In [131]:
# from Analysis1_question_userid import user_ids
bronze = 0 
silver = 0
gold = 0
user_badge_dict={}
user_ids_distinct = list(set(user_ids))
div = len(user_ids_distinct)//25
for i in range(0,div):
    userid_str = ";".join(str(x) for x in user_ids_distinct[0+i*25:25+i*25])
    key = '1N2)WAQaEFzyJP*F)Al2Kw(('
    url = 'http://api.stackexchange.com/2.2/users/{}?key={}\
    &order=desc&sort=reputation&site=stackoverflow'.format(userid_str,key)
    r = requests.get(url)
    items = r.json()["items"]
    for item in items:
        user_id = item["user_id"]
        user_bronze = item["badge_counts"]["bronze"]
        user_silver = item["badge_counts"]["silver"]
        user_gold = item["badge_counts"]["gold"]

        user_badge_dict[user_id]= [user_bronze, user_silver, user_gold]

        bronze += user_bronze
        silver += user_silver
        gold += user_gold
# print(len(user_badge_dict))
# print("end")

In [132]:
rows = list()
for ques,user_id in ques_user_dict.items():
    u_bronze = user_badge_dict[user_id][0]
    u_silver = user_badge_dict[user_id][1]
    u_gold = user_badge_dict[user_id][2]
    rows.append((user_id, float(format(u_bronze/bronze + u_silver/silver + u_gold/gold,'.6f')),
                     u_bronze, u_silver, u_gold, ques))
rows_sorted = sorted(rows, key = lambda x: x[1],reverse=True)

with open('badges_weightage.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["user_id", "weightage", "bronze", "silver", "gold", "question title"])
    writer.writerows(rows_sorted)
    print(len(rows_sorted))

600
